<a href="https://colab.research.google.com/github/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/blob/main/NFLSpreadAndLineOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project to predict future NFL Spread & Line outcomes

## Author: Tito Yslas

### Background
I enjoy watching the NFL and playing fantasy football. I also like to place bets on games using apps like FanDuel. The purpose of this project is to increase my understanding of the NFL betting market and possibly create a machine learning model to give myself an edge next season.

### Project Description
I found a dataset from Kaggle titled [NFL scores and betting data](https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?resource=download). This dataset has over 13,000 samples and 17 features. The goal is to use this dataset to train a model that will predict the winner of a given game.

### Performance Metric
For my performance metric, I will aim for the model to have at least 70% accuracy.

## Import Libraries

In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

### Data Dictionary
- schedule_date: Date that the game took place. This column is a date in the format MM/DD/YYYY
- schedule_season: Year of the season began. NFL seasons start in the fall and end before the spring of the following year. So the 2023 season will refer to the years 2023-2024. This column is a number in the format YYYY
- schedule_week: Week of the NFL season. This column is either a number during regular season weeks or a string in playoff weeks. For the purposes of this project the playoff weeks will be converted to numbers and the schedule_playoff column will be used to determine whether the week is a playoff game or not
- schedule_playoff: This column is a boolean. FALSE is regular season and TRUE is playoffs
- team_home: Name of the home team. This column is a string
- score_home: Points scored by the home team. This column is a number
- score_away: Points scored by the away team. This column is a number
- winner: This column will be a feature derived from score_home and score_away columns to that will use one hot encoding - if team_home scores more points this will be a 1 - if team_home scores fewer points it will be a 0
- team_away: Name of the away team. This column is a string
- team_favorite_id: Acronym of the team that was determined most likely to win by the betting market. It is either two or three letters. For the purposes of this project this column will be changed to be either the team_home or team_away name
- team_home_favorite: this will represent the encoded team_favorite_id - if team_home is favored this column will be marked as a 1 - if it's a zero then we know that team_away is favored
- spread_favorite: The number of points that the favored team needs to win by for a bet placed on the spread of the favorite to win. This column will either be a negative number or zero
- over_under_line: The number of points that both teams combined need to score for a bet placed on the 'line' to win. This column is a positive number
- stadium: Name of the venue that the game is played
- stadium_neutral: This column is a boolean. FALSE is not a neutral venue and TRUE is a neutral venue - for the purposes of this project this column will be one hot encoded with a neutral venue being marked as a 1 and non-neutral marked as a 0
- weather_temperature: The temperature in Fahrenheit at the venue where the game is played. This column is a number
- weather_wind_mph: The speed of wind in miles per hour. This column is a number
- weather_humidity: The measurement of water vapor in the air during the game measured as a percentage. This column is a number
- weather_detail: Other information about the weather conditions - if the venue is indoor or the venue has a retractable roof. This column is a string


## Exploratory Data Analysis
### Questions to answer with EDA:
1. Which columns, if any, should I modify the data type to better train my model?
1. Which columns, if any, should I remove from the training and test data so that the model can be effectively trained?
1. Which columns, if any, should I remove or insert derived data for in the case that there is a lot of missing data?
1. What features could it make sense to introduce to improve the training and performance of my model?

In [2]:
# print all file names in directory
!wget -O spreadspoke_scores.csv https://raw.githubusercontent.com/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/main/spreadspoke_scores.csv
!wget -O data_dictionary.csv https://raw.githubusercontent.com/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/main/data_dictionary.csv
!wget -O team_ids.py https://raw.githubusercontent.com/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/main/team_ids.py
for file in os.listdir():
  print(file)

--2023-06-19 18:32:24--  https://raw.githubusercontent.com/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/main/spreadspoke_scores.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1441380 (1.4M) [text/plain]
Saving to: ‘spreadspoke_scores.csv’

spreadspoke_scores. 100%[===================>]   1.37M  --.-KB/s    in 0.05s   

2023-06-19 18:32:25 (25.1 MB/s) - ‘spreadspoke_scores.csv’ saved [1441380/1441380]

--2023-06-19 18:32:25--  https://raw.githubusercontent.com/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/main/data_dictionary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HT

In [3]:
scores = pd.read_csv('spreadspoke_scores.csv')

In [4]:
scores.info()
# scores.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13516 entries, 0 to 13515
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        13516 non-null  object 
 1   schedule_season      13516 non-null  int64  
 2   schedule_week        13516 non-null  object 
 3   schedule_playoff     13516 non-null  bool   
 4   team_home            13516 non-null  object 
 5   score_home           13516 non-null  int64  
 6   score_away           13516 non-null  int64  
 7   team_away            13516 non-null  object 
 8   team_favorite_id     11037 non-null  object 
 9   spread_favorite      11037 non-null  float64
 10  over_under_line      11027 non-null  object 
 11  stadium              13516 non-null  object 
 12  stadium_neutral      13516 non-null  bool   
 13  weather_temperature  12309 non-null  float64
 14  weather_wind_mph     12293 non-null  float64
 15  weather_humidity     8468 non-null  

### Answer to Question 1
- I think that it makes sense to modify both team_home and team_away to have the same IDs as team_favorite_id to make it easier for the model to indentify when team_home is the same or different as team_favorite_id
- Currently Pandas is indentifying schedule_date as an object, it could make sense to see if there's a way for Pandas to indentify this as a date
- Currently Pandas is indentifying schedule_week as an object. This is because some of the data in this column is in string format. It could make sense to modify the data of this column that is in a string to only be in int64 format and use the schedule_playoff column to be the soe determination of whether or not the schedule_week is a playoff game
- Currently Pandas is indentifying the over_under_line column as an object data type despite the fact that it should be a float. I will explore how to ensure that this column's data type is correctly identified
- The stadium_neutral column is currently a boolean type, I think I will convert this to use one hot encoding instead

### Answer to Question 2
- Based on my initial examination, I'm not sure if it makes sense to remove any of my columns from the data set on which I will train my model

In [5]:
scores.isna().sum() # number of missing values for each column

schedule_date              0
schedule_season            0
schedule_week              0
schedule_playoff           0
team_home                  0
score_home                 0
score_away                 0
team_away                  0
team_favorite_id        2479
spread_favorite         2479
over_under_line         2489
stadium                    0
stadium_neutral            0
weather_temperature     1207
weather_wind_mph        1223
weather_humidity        5048
weather_detail         10597
dtype: int64

### Answer to Question 3
- the columns with missing data include team_favorite_id, spread_favorite, over_under_line, weather_temperature, weather_wind_mph, weather_humidity, and weather_detail. these columns have many missing values because this data was not collected in earlier seasons. for example, there is minimal team_favorite_id information collected from the 1978 schedule_season and previous to that likely because of the lack of public betting information before that time
- for the missing data I don't think that it makes sense to remove these columns, however it might make sense to only train and test on the observations from the 1979 schedule_season and beyond

### Answer to Question 4
- I think it makes sense to introduce/derive three different target columns for understanding the performance of the model
- The three target columns I am thinking about introducing are derived from score_home, score_away, and over_under_line
- These targets would be one hot encoded as team_home_win, team_home_cover_spread, and cover_line
- team_home_win would be a 1 if team_home wins or a 0 if they lose
- team_home_cover_spread would be a 1 if they cover the spread_favorite and a 0 if they don't
- cover_line would be a 1 if score_home + score_way is greater than the over_under_line and a 0 if it's less than

## Feature Engineering
1. convert `schedule_date` column to actually be read in as a date/time object instead of generic object
1. convert `schedule_week` to only be numbers - this could be more challenging than I initially thought because there will not be direct mappings for the outliers of `Division`, `Wild Card`, `Conference`, and `Superbowl` because the NFL has expanded the numbers of games played during the regular season over the years and added in the `Wildcard` games
1. convert the `schedule_playoff` column from true/false to 1/0
1. add a target/feature of `winning_team` to be derived from `score_home` and `score_away` to make it easier to determine how the model performs
1. convert all `team_home` and `team_away` entries to the acronym identifiers
1. drop rows 0 - 2499 because they don't have data for `team_favorite_id`, `spread_favorite` and `over_under_line`
1. create new target column of `favorite_won` with 1 for true and 0 for false
1. create new target column of `spread_favorite_covered` with 1 for true and 0 for false
1. create new target column of `over_under_covered`
1. convert the `weather_detail` column to a categorical variable that is one-hot encoded

In [6]:
# convert schedule_date to proper date type
scores['schedule_date'] = pd.to_datetime(scores['schedule_date'])
schedule_date_data_type = scores['schedule_date'].dtype
print('data type of schedule_date: ', schedule_date_data_type)

data type of schedule_date:  datetime64[ns]


In [7]:
# convert schedule_week to only be numbers
scores['schedule_week'].value_counts()
# this may require a fair amount of time spent manually mapping week numbers

2             831
13            829
1             826
14            826
12            825
11            808
3             782
10            772
4             757
9             754
7             752
8             751
5             745
6             740
15            701
16            687
17            527
Division      217
Wildcard      168
Conference    115
Superbowl      57
18             46
Name: schedule_week, dtype: int64

In [8]:
# convert the schedule_playoff column from true/false into 1/0
scores['schedule_playoff'] = scores['schedule_playoff'].astype(int)
display(scores)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,2066-09-02,1966,1,0,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,2066-09-03,1966,1,0,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,2066-09-04,1966,1,0,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,2066-09-09,1966,2,0,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,2066-09-10,1966,1,0,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13511,2023-01-22,2022,Division,1,Buffalo Bills,10,27,Cincinnati Bengals,BUF,-6.0,48.5,Highmark Stadium,False,32.0,4.0,100.0,snow
13512,2023-01-22,2022,Division,1,San Francisco 49ers,19,12,Dallas Cowboys,SF,-3.5,46.5,Levi's Stadium,False,55.0,19.0,47.0,NaN
13513,2023-01-29,2022,Conference,1,Kansas City Chiefs,23,20,Cincinnati Bengals,KC,-1.5,48,GEHA Field at Arrowhead Stadium,False,22.0,13.0,55.0,NaN
13514,2023-01-29,2022,Conference,1,Philadelphia Eagles,31,7,San Francisco 49ers,PHI,-2.5,45.5,Lincoln Financial Field,False,52.0,14.0,48.0,rain


In [9]:
# convert all team_home and team_away entries to the acronym identifiers

# map of team names and their corresponding IDs
team_ids = {
    'San Francisco 49ers': 'SF',
    'Dallas Cowboys': 'DAL',
    'Pittsburgh Steelers': 'PIT',
    'Green Bay Packers': 'GB',
    'Philadelphia Eagles': 'PHI',
    'Minnesota Vikings': 'MIN',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Kansas City Chiefs': 'KC',
    'Buffalo Bills': 'BUF',
    'Chicago Bears': 'CHI',
    'New York Giants': 'NYG',
    'Atlanta Falcons': 'ATL',
    'New Orleans Saints': 'NO',
    'New York Jets': 'NYJ',
    'Detroit Lions': 'DET',
    'Cincinnati Bengals': 'CIN',
    'New England Patriots': 'NE',
    'Washington Redskins': 'WAS',
    'Cleveland Browns': 'CLE',
    # should be SD but all the data uses LAC
    'San Diego Chargers': 'LAC',
    'Seattle Seahawks': 'SEA',
    'Tampa Bay Buccaneers': 'TB',
    # should be OAK but all the data uses LVR
    'Oakland Raiders': 'LVR',
    'Indianapolis Colts': 'IND',
    'Los Angeles Rams': 'LAR',
    'Arizona Cardinals': 'ARI',
    # should be HOU but all the data uses TEN
    'Houston Oilers': 'TEN',
    'Carolina Panthers': 'CAR',
    'Jacksonville Jaguars': 'JAX',
    'Baltimore Ravens': 'BAL',
    'Tennessee Titans': 'TEN',
    # should be STL but all the data uses LAR
    'St. Louis Rams': 'LAR',
    'Houston Texans': 'HOU',
    # should be STL but all the data uses ARI
    'St. Louis Cardinals': 'ARI',
    'Baltimore Colts': 'BAL',
    # should be LAR but all the data uses LVR
    'Los Angeles Raiders': 'LVR',
    'Los Angeles Chargers': 'LAC',
    # should be PHX but all the data uses ARI
    'Phoenix Cardinals': 'ARI',
    # 'Boston Patriots': '', the franchise changed the name of their team to the New England Patriots in 1971
    # the data does not have any team_favorite_id listed before the 12/24/78 in the 1978 season (row 2494)
    'Las Vegas Raiders': 'LVR',
    'Washington Football Team': 'WAS',
    'Tennessee Oilers': 'TEN',
    'Washington Commanders': 'WAS',
}

scores['team_home'] = scores['team_home'].replace(team_ids)
scores['team_away'] = scores['team_away'].replace(team_ids)

scores.loc[2492:2502]

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
2492,1978-12-24,1978,Wildcard,1,ATL,14,13,PHI,ATL,-2.5,NaN,Atlanta-Fulton County Stadium,False,43.0,11.0,77.0,NaN
2493,1978-12-24,1978,Wildcard,1,MIA,9,17,TEN,MIA,-6.5,NaN,Orange Bowl,False,77.0,12.0,78.0,NaN
2494,1978-12-30,1978,Division,1,DAL,27,20,ATL,DAL,-15.0,NaN,Texas Stadium,False,38.0,15.0,97.0,NaN
2495,1978-12-30,1978,Division,1,PIT,33,10,DEN,PIT,-7.0,NaN,Three Rivers Stadium,False,30.0,7.0,75.0,NaN
2496,1978-12-31,1978,Division,1,LAR,34,10,MIN,LAR,-7.5,NaN,Los Angeles Memorial Coliseum,False,53.0,6.0,52.0,NaN
2497,1978-12-31,1978,Division,1,NE,14,31,TEN,NE,-6.0,NaN,Foxboro Stadium,False,36.0,8.0,70.0,NaN
2498,1979-01-07,1978,Conference,1,LAR,0,28,DAL,DAL,-3.5,NaN,Los Angeles Memorial Coliseum,False,56.0,8.0,77.0,NaN
2499,1979-01-07,1978,Conference,1,PIT,34,5,TEN,PIT,-7.0,NaN,Three Rivers Stadium,False,25.0,8.0,85.0,NaN
2500,1979-01-21,1978,Superbowl,1,DAL,31,35,PIT,PIT,-3.5,37,Orange Bowl,True,71.0,18.0,84.0,rain
2501,1979-09-01,1979,1,0,TB,31,16,DET,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN


In [10]:
# add a target/feature of 'winning_team'
# Derive a new column based on the comparison of two existing columns
def determine_winner(row):
  if row['score_home'] > row['score_away']:
    val = row['team_home']
  elif row['score_home'] == row['score_away']:
    val = 'tie'
  else:
    val = row['team_away']
  return val

scores['winning_team'] = scores.apply(determine_winner, axis=1)
print(scores['winning_team'].tail(20))
print('number of ties:', scores['winning_team'].value_counts()['tie'])

13496    MIA
13497    CAR
13498    PHI
13499    PIT
13500     SF
13501    SEA
13502    WAS
13503    JAX
13504     SF
13505    BUF
13506    CIN
13507    NYG
13508    DAL
13509     KC
13510    PHI
13511    CIN
13512     SF
13513     KC
13514    PHI
13515     KC
Name: winning_team, dtype: object
number of ties: 91


In [11]:
# drop rows 0 - 2499 because they don't have data for team_favorite_id, spread_favorite and over_under_line
start_index = 0
end_index = 2500

scores_dropped = scores.drop(scores.index[start_index:end_index])
scores_dropped.reset_index(drop = True, inplace = True)
scores_dropped.head(20)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team
0,1979-01-21,1978,Superbowl,1,DAL,31,35,PIT,PIT,-3.5,37,Orange Bowl,True,71.0,18.0,84.0,rain,PIT
1,1979-09-01,1979,1,0,TB,31,16,DET,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN,TB
2,1979-09-02,1979,1,0,BUF,7,9,MIA,MIA,-5.0,39,Ralph Wilson Stadium,False,74.0,15.0,74.0,NaN,MIA
3,1979-09-02,1979,1,0,CHI,6,3,GB,CHI,-3.0,31,Soldier Field,False,78.0,11.0,68.0,NaN,CHI
4,1979-09-02,1979,1,0,DEN,10,0,CIN,DEN,-3.0,31.5,Mile High Stadium,False,69.0,6.0,38.0,NaN,DEN
5,1979-09-02,1979,1,0,KC,14,0,BAL,KC,-1.0,37,Arrowhead Stadium,False,76.0,8.0,71.0,NaN,KC
6,1979-09-02,1979,1,0,LAR,17,24,LVR,LAR,-4.0,36.5,Anaheim Stadium,False,70.0,10.0,77.0,NaN,LVR
7,1979-09-02,1979,1,0,MIN,28,22,SF,MIN,-7.0,32,Metropolitan Stadium,False,70.0,11.0,67.0,NaN,MIN
8,1979-09-02,1979,1,0,NO,34,40,ATL,NO,-5.0,32,Louisiana Superdome,False,72.0,0.0,NaN,indoor,ATL
9,1979-09-02,1979,1,0,NYJ,22,25,CLE,NYJ,-2.0,41,Giants Stadium,False,73.0,10.0,76.0,NaN,CLE


In [12]:
# create new target column of favorite_won with 1 for true and 0 for false
# the favorite_won column is returning an float instead of an int -- need to fix
# scores_dropped.drop('favorite_won', inplace = True)

def determine_favorite_won(row):
  result = 1
  if row['winning_team'] != row['team_favorite_id']:
    result = 0
  return int(result)

scores_dropped['favorite_won'] = scores_dropped.apply(determine_favorite_won, axis = 1)
scores_dropped.loc['favorite_won'] = scores_dropped['favorite_won'].astype(int)
scores_dropped.head()

<ipython-input-12-8cf4bdcaf6d8>:12: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  scores_dropped.loc['favorite_won'] = scores_dropped['favorite_won'].astype(int)


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team,favorite_won
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,37,Orange Bowl,1.0,71.0,18.0,84.0,rain,PIT,1.0
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,30,Houlihan's Stadium,0.0,79.0,9.0,87.0,NaN,TB,1.0
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,39,Ralph Wilson Stadium,0.0,74.0,15.0,74.0,NaN,MIA,1.0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,31,Soldier Field,0.0,78.0,11.0,68.0,NaN,CHI,1.0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,31.5,Mile High Stadium,0.0,69.0,6.0,38.0,NaN,DEN,1.0


In [17]:
# create new target column of spread_favorite_covered with 1 for true and 0 for false
# the favorite_won column is returning an float instead of an int -- need to fix

def determine_favorite_covered_spread(row):
  return 1 if row['score_home'] - row['score_away'] > abs(row['spread_favorite']) else 0

scores_dropped['spread_favorite_covered'] = scores_dropped.apply(determine_favorite_covered_spread, axis = 1)

counts = scores_dropped['spread_favorite_covered'].value_counts()
favorite_missed = counts.values[0]
favorite_covered = counts.values[1]
percent_favorite_won = round((favorite_covered / (favorite_covered + favorite_missed)) * 100, 2)

print(f"the favorite team covered the spread {percent_favorite_won}% of the time")

display(scores_dropped.head(20))

the favorite team covered the spread 40.52% of the time


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team,favorite_won,spread_favorite_covered,over_under_covered
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,...,Orange Bowl,1.0,71.0,18.0,84.0,rain,PIT,1.0,0,1
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,...,Houlihan's Stadium,0.0,79.0,9.0,87.0,NaN,TB,1.0,1,1
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,...,Ralph Wilson Stadium,0.0,74.0,15.0,74.0,NaN,MIA,1.0,0,0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,...,Soldier Field,0.0,78.0,11.0,68.0,NaN,CHI,1.0,0,0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,...,Mile High Stadium,0.0,69.0,6.0,38.0,NaN,DEN,1.0,1,0
5,1979-09-02,1979.0,1,0.0,KC,14.0,0.0,BAL,KC,-1.0,...,Arrowhead Stadium,0.0,76.0,8.0,71.0,NaN,KC,1.0,1,0
6,1979-09-02,1979.0,1,0.0,LAR,17.0,24.0,LVR,LAR,-4.0,...,Anaheim Stadium,0.0,70.0,10.0,77.0,NaN,LVR,0.0,0,1
7,1979-09-02,1979.0,1,0.0,MIN,28.0,22.0,SF,MIN,-7.0,...,Metropolitan Stadium,0.0,70.0,11.0,67.0,NaN,MIN,1.0,0,1
8,1979-09-02,1979.0,1,0.0,NO,34.0,40.0,ATL,NO,-5.0,...,Louisiana Superdome,0.0,72.0,0.0,NaN,indoor,ATL,0.0,0,1
9,1979-09-02,1979.0,1,0.0,NYJ,22.0,25.0,CLE,NYJ,-2.0,...,Giants Stadium,0.0,73.0,10.0,76.0,NaN,CLE,0.0,0,1


In [16]:
# create new target column of over_under_covered

# need to cast over_under_line column to a float -- pandas recognizes it as a string
scores_dropped['over_under_line'] = pd.to_numeric(scores_dropped['over_under_line'], errors='coerce')
# scores_dropped['over_under_line'].astype(float)
print(scores_dropped['over_under_line'].dtypes)
def determine_over_under_covered(row):
  return 1 if row['score_home'] + row['score_away'] > row['over_under_line'] else 0

scores_dropped['over_under_covered'] = scores_dropped.apply(determine_over_under_covered, axis = 1)

counts = scores_dropped['over_under_covered'].value_counts()
over_missed = counts.values[0]
over_covered = counts.values[1]
percent_over_covered = round((over_covered / (over_covered + over_missed)) * 100, 2)

print(f"the over hit {percent_over_covered}% of the time")

display(scores_dropped[['score_home', 'score_away', 'over_under_line', 'over_under_covered']].head(20))

float64
the over hit 47.96% of the time


,score_home,score_away,over_under_line,over_under_covered
0,31.0,35.0,37.0,1
1,31.0,16.0,30.0,1
2,7.0,9.0,39.0,0
3,6.0,3.0,31.0,0
4,10.0,0.0,31.5,0
5,14.0,0.0,37.0,0
6,17.0,24.0,36.5,1
7,28.0,22.0,32.0,1
8,34.0,40.0,32.0,1
9,22.0,25.0,41.0,1


In [18]:
# convert the weather_detail column to categorical variable that is one-hot encoded

counts = scores_dropped['weather_detail'].value_counts()

totalWeatherDetails = 0
for count in counts:
  totalWeatherDetails += count
print(f"total number of rows with weather details: {totalWeatherDetails}")
numRows = scores_dropped.shape[0]
print(f"number of rows: {numRows}")

one_hot_encoded = pd.get_dummies(scores_dropped['weather_detail'], prefix = 'conditions')
scores_encoded = pd.concat([scores_dropped, one_hot_encoded], axis = 1)

scores_encoded.head(20)

total number of rows with weather details: 2758
number of rows: 11017


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,spread_favorite_covered,over_under_covered,conditions_fog,conditions_indoor,conditions_rain,conditions_rain | fog,conditions_retractable (open roof),conditions_snow,conditions_snow | Freezing rain,conditions_snow | fog
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,...,0,1,0,0,1,0,0,0,0,0
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,...,1,1,0,0,0,0,0,0,0,0
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,...,0,0,0,0,0,0,0,0,0,0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,...,0,0,0,0,0,0,0,0,0,0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,...,1,0,0,0,0,0,0,0,0,0
5,1979-09-02,1979.0,1,0.0,KC,14.0,0.0,BAL,KC,-1.0,...,1,0,0,0,0,0,0,0,0,0
6,1979-09-02,1979.0,1,0.0,LAR,17.0,24.0,LVR,LAR,-4.0,...,0,1,0,0,0,0,0,0,0,0
7,1979-09-02,1979.0,1,0.0,MIN,28.0,22.0,SF,MIN,-7.0,...,0,1,0,0,0,0,0,0,0,0
8,1979-09-02,1979.0,1,0.0,NO,34.0,40.0,ATL,NO,-5.0,...,0,1,0,1,0,0,0,0,0,0
9,1979-09-02,1979.0,1,0.0,NYJ,22.0,25.0,CLE,NYJ,-2.0,...,0,1,0,0,0,0,0,0,0,0


## Train/Test/Hold-Out Split

In [19]:
# machine learning libs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [26]:
# remove team_home and team_away columns from data so that the model cannot have access to outcome information
# dropping multiple columns
columns_to_drop = ['score_home', 'score_away']
prepared_data = scores_encoded.drop(columns_to_drop, axis = 1)

# multiple targets: y1 = favorite_won, y2 = spread_favorite_covered, y3 = over_under_covered
# 'X' is the feature matrix and 'y1', 'y2', 'y3' are the target variables
X = prepared_data
y1 = prepared_data['favorite_won']
y2 = prepared_data['spread_favorite_covered']
y3 = prepared_data['over_under_covered']
test_percent = 0.25
random_seed = 42
holdout_percent = 0.5

# split the data into train and combined test-holdout sets (75% train, 25% test + holdout)
X_train, X_test_holdout, y1_train, y1_test_holdout, y2_train, y2_test_holdout, y3_train, y3_test_holdout = train_test_split(X, y1, y2, y3, test_size = test_percent, random_state = random_seed)

# further split the test-holdout set into test and holdout sets (50% test, 50% holdout)
X_test, X_holdout, y1_test, y1_holdout, y2_test, y2_holdout, y3_test, y3_holdout = train_test_split(X_test_holdout, y1_test_holdout, y2_test_holdout, y3_test_holdout, test_size = holdout_percent, random_state = random_seed)


# X train
print('On X train:')
print(f'X train dimensions: {X_train.shape}')
display(X_train.head())

# X test
print('\nOn X test:')
print(f'X test dimensions: {X_test.shape}')
display(X_test.head())

# X holdout
print('\nOn X holdout:')
print(f'X holdout dimensions: {X_holdout.shape}')
display(X_holdout.head())


On X train: 
X train dimensions: (8262, 27)


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,...,spread_favorite_covered,over_under_covered,conditions_fog,conditions_indoor,conditions_rain,conditions_rain | fog,conditions_retractable (open roof),conditions_snow,conditions_snow | Freezing rain,conditions_snow | fog
5317,2001-12-22,2001.0,15,0.0,LVR,TEN,LVR,-6.0,45.5,Oakland Coliseum,...,0,0,0,0,0,0,0,0,0,0
7049,2008-10-12,2008.0,6,0.0,ATL,CHI,CHI,-3.0,43.5,Georgia Dome,...,0,0,0,1,0,0,0,0,0,0
8525,2013-12-15,2013.0,15,0.0,MIN,PHI,PHI,-7.0,53.0,Mall of America Field,...,1,1,0,1,0,0,0,0,0,0
994,1983-11-14,1983.0,11,0.0,ATL,LAR,ATL,-1.5,42.0,Atlanta-Fulton County Stadium,...,0,1,0,0,0,0,0,0,0,0
3753,1995-11-19,1995.0,12,0.0,TB,JAX,TB,-7.0,37.0,Houlihan's Stadium,...,0,0,0,0,0,0,0,0,0,0



On X test: 
X test dimensions: (1377, 27)


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,...,spread_favorite_covered,over_under_covered,conditions_fog,conditions_indoor,conditions_rain,conditions_rain | fog,conditions_retractable (open roof),conditions_snow,conditions_snow | Freezing rain,conditions_snow | fog
10142,2019-12-22,2019.0,16,0.0,IND,CAR,IND,-7.0,46.5,Lucas Oil Stadium,...,1,0,0,1,0,0,0,0,0,0
9180,2016-10-09,2016.0,5,0.0,IND,CHI,IND,-4.0,47.5,Lucas Oil Stadium,...,1,1,0,1,0,0,0,0,0,0
259,1980-09-14,1980.0,2,0.0,LAC,LVR,LAC,-7.0,44.0,Qualcomm Stadium,...,0,1,0,0,0,0,0,0,0,0
1950,1988-09-04,1988.0,1,0.0,BUF,MIN,MIN,-2.5,44.0,Ralph Wilson Stadium,...,1,0,0,0,0,0,0,0,0,0
4826,1999-12-26,1999.0,16,0.0,TEN,JAX,JAX,-3.0,39.0,Nissan Stadium,...,1,1,0,0,0,0,0,0,0,0



On X holdout: 
X holdout dimensions: (1378, 27)


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,...,spread_favorite_covered,over_under_covered,conditions_fog,conditions_indoor,conditions_rain,conditions_rain | fog,conditions_retractable (open roof),conditions_snow,conditions_snow | Freezing rain,conditions_snow | fog
4575,1998-12-27,1998.0,17,0.0,CIN,TB,TB,-6.5,42.0,Cinergy Field,...,0,0,0,0,0,0,0,0,0,0
4802,1999-12-19,1999.0,15,0.0,CHI,DET,DET,-3.0,37.0,Soldier Field,...,1,1,0,0,0,0,0,0,0,0
467,1981-09-05,1981.0,1,0.0,TB,MIN,TB,-1.0,37.0,Houlihan's Stadium,...,1,0,0,0,0,0,0,0,0,0
3050,1992-11-29,1992.0,13,0.0,LAC,LVR,LAC,-3.5,35.0,Qualcomm Stadium,...,1,0,0,0,0,0,0,0,0,0
334,1980-10-26,1980.0,8,0.0,BUF,NE,NE,-2.0,42.0,Ralph Wilson Stadium,...,1,1,0,0,0,0,0,0,0,0


In [32]:
# y1 = favorite_won, y2 = spread_favorite_covered, y3 = over_under_covered

# favorite_won targets
# y1 train
print('On y1 train (favorite won):')
print(f'y1 train dimensions: {y1_train.shape}')
display(y1_train.head())

# y1 test
print('\nOn y1 test (favorite won):')
print(f'y1 test dimensions: {y1_test.shape}')
display(y1_test.head())

# y1 holdout
print('\nOn y1 holdout (favorite won):')
print(f'y1 holdout dimensions: {y1_holdout.shape}')
display(y1_holdout.head())

# spread_favorite_covered targets
# y2 train
print('\nOn y2 train (spread favorite covered):')
print(f'y2 train dimensions: {y2_train.shape}')
display(y2_train.head())

# y2 test
print('\nOn y2 test (spread favorite covered):')
print(f'y2 test dimensions: {y2_test.shape}')
display(y2_test.head())

# y2 holdout
print('\nOn y2 holdout (spread favorite covered):')
print(f'y2 holdout dimensions: {y2_holdout.shape}')
display(y2_holdout.head())

# over_under_covered targets
# y3 train
print('\nOn y3 train (over/under covered):')
print(f'y3 train dimensions: {y3_train.shape}')
display(y3_train.head())

# y3 test
print('\nOn y3 test (over/under covered):')
print(f'y3 test dimensions: {y3_test.shape}')
display(y3_test.head())

# y3 holdout
print('\nOn y3 holdout (over/under covered):')
print(f'y3 holdout dimensions: {y3_holdout.shape}')
display(y3_holdout.head())


On y1 train (favorite won):
y1 train dimensions: (8262,)


5317    0.0
7049    0.0
8525    0.0
994     0.0
3753    1.0
Name: favorite_won, dtype: float64


On y1 test (favorite won):
y1 test dimensions: (1377,)


10142    1.0
9180     1.0
259      1.0
1950     0.0
4826     0.0
Name: favorite_won, dtype: float64


On y1 holdout (favorite won):
y1 holdout dimensions: (1378,)


...

5317    0
7049    0
8525    1
994     1
3753    0
Name: over_under_covered, dtype: int64


On y3 test (over/under covered):
y3 test dimensions: (1377,)


10142    0
9180     1
259      1
1950     0
4826     1
Name: over_under_covered, dtype: int64


On y3 holdout (over/under covered):
y3 holdout dimensions: (1378,)


4575    0
4802    1
467     0
3050    0
334     1
Name: over_under_covered, dtype: int64

## Establish Training Pipeline